In [1]:
import numpy as np
%matplotlib qt
import matplotlib.pyplot as plt
from tqdm import tqdm
from skimage import io, color
from skimage.morphology import binary_erosion, square
import cv2

In [2]:
# Import a few sample images
img1 = io.imread('Extracted Characters/sample1.png')
img2 = io.imread('Extracted Characters/sample2.png')
img3 = io.imread('Extracted Characters/sample3.png')
print(f'img1.shape: {img1.shape}')
print(f'img2.shape: {img2.shape}')
print(f'img3.shape: {img3.shape}')
print(f'img1.dtype: {img1.dtype}')
print(f'img2.dtype: {img2.dtype}')
print(f'img3.dtype: {img3.dtype}')

img1.shape: (350, 462, 3)
img2.shape: (292, 333, 3)
img3.shape: (356, 311, 3)
img1.dtype: uint8
img2.dtype: uint8
img3.dtype: uint8


# Preprocessing

In [3]:
# Scale and pad

# Based on tutorial: https://jdhao.github.io/2017/11/06/resize-image-to-square-with-padding/
def make_square(img, desired_size=256, fill_color=[255, 255, 255]):
    if img.dtype != np.uint8:
        print(f'Converting to uint8...')
        img = (255*img).astype(np.uint8)
        
    scale_factor = desired_size/max(img.shape[0], img.shape[1])
    resized = cv2.resize(img, (int(scale_factor*img.shape[1]), int(scale_factor*img.shape[0])))
    new_size = resized.shape
    
    delta_w = desired_size - new_size[1]
    delta_h = desired_size - new_size[0]
    top, bottom = delta_h//2, delta_h-(delta_h//2)
    left, right = delta_w//2, delta_w-(delta_w//2)
    
    out = cv2.copyMakeBorder(resized, top, bottom, left, right, cv2.BORDER_CONSTANT, value=fill_color)
    return out

img1_square = make_square(img1)
img2_square = make_square(img2)
img3_square = make_square(img3)
print(f'img1_square.dtype: {img1_square.dtype}')
print(f'img1_square.shape: {img1_square.shape}')
print(f'img2_square.shape: {img2_square.shape}')
print(f'img3_square.shape: {img3_square.shape}')

img1_square.dtype: uint8
img1_square.shape: (256, 256, 3)
img2_square.shape: (256, 256, 3)
img3_square.shape: (256, 256, 3)


In [4]:
# Binarize
# use float for thresholding, but return uint8 image
def binarize(img, threshold=0.5, invert=True):
    if img.dtype == np.uint8:
        img = img/255.0 # convert to float64

    # Convert to grayscale
    if len(img.shape) >= 3:
        img = color.rgb2gray(img)
    
    # Threshold
    out = np.zeros_like(img)
    if invert: # detect dark characters
        mask = img < threshold
    else: # detect light characters
        mask = img > threshold
    out[mask] = 1
    return (255*out).astype(np.uint8)

img1_bin = binarize(img1_square)
img2_bin = binarize(img2_square)
img3_bin = binarize(img3_square)
print(f'img1_bin.dtype: {img1_bin.dtype}')
print(f'img1_bin.shape: {img1_bin.shape}')
print(f'img1_bin.max() = {img1_bin.max()}')
plt.imshow(img1_bin, cmap='gray')

img1_bin.dtype: uint8
img1_bin.shape: (256, 256)
img1_bin.max() = 255


In [5]:
num_rows = 3
num_cols = 3
fig, axes = plt.subplots(num_rows, num_cols, figsize=(4*num_cols, 4*num_rows))
for i, img in enumerate([img1, img2, img3]):
    axes[0, i].imshow(img)
    axes[0, i].set_title(f'Sample {i+1}')
for i, img in enumerate([img1_square, img2_square, img3_square]):
    axes[1, i].imshow(img)
    axes[1, i].set_title(f'Scaled & padded {i+1}')
for i, img in enumerate([img1_bin, img2_bin, img3_bin]):
    axes[2, i].imshow(img, cmap='gray')
    axes[2, i].set_title(f'Binarized {i+1}')
    
plt.suptitle('Pre-processing Samples', fontsize='xx-large')
plt.tight_layout()
plt.savefig('PreprocessingSamples.png')

# Feature Extraction (Contour Analysis)

In [6]:
img1_bin.shape

(256, 256)

In [7]:
img1_bin.dtype

dtype('uint8')

In [8]:
# Example from tutorial
import numpy as np
import cv2 as cv
import matplotlib

im = cv.imread('test.jpg')
imgray = cv.cvtColor(im, cv.COLOR_BGR2GRAY)
ret, thresh = cv.threshold(imgray, 127, 255, 0)
contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)

print(f'im.shape: {im.shape}')
print(f'imgray.shape: {imgray.shape}')
print(f'imgray.dtype: {imgray.dtype}')
print(ret)
print(f'thresh.shape: {thresh.shape}')
print(f'thresh.dtype: {thresh.dtype}')
print(f'thresh.max() = {thresh.max()}')
print(type(contours))
print(f'# of contours: {len(contours)}')

dst = im.copy()
cmap = matplotlib.cm.get_cmap('tab10')
for i, cnt in enumerate(contours):
    cv.drawContours(dst, [cnt], 0, [255*x for x in cmap(i)], 3)

plt.imshow(dst)
plt.title('Test image with labeled contours')
plt.savefig('ContourTest.png')

im.shape: (350, 462, 3)
imgray.shape: (350, 462)
imgray.dtype: uint8
127.0
thresh.shape: (350, 462)
thresh.dtype: uint8
thresh.max() = 255
<class 'list'>
# of contours: 9


In [9]:
# Extract contours
im = img1_bin

# Note: findContours requires a binarized image with white shape on black background, and type uint8
contours, hierarchy = cv2.findContours(im, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
print(f'# of contours: {len(contours)}')
print('# of points in each contour:')
for cnt in contours:
    print(f'\t{len(cnt)} points')

dst = img1_square.copy()
cmap = matplotlib.cm.get_cmap('tab10')
for i, cnt in enumerate(contours):
    cv2.drawContours(dst, [cnt], 0, [255*x for x in cmap(i)], 3)

plt.figure(figsize=(4, 4))
plt.imshow(dst)
plt.title('Sample 1 with labeled contours')
plt.savefig('ContoursSample1.png')

# of contours: 5
# of points in each contour:
	1 points
	103 points
	97 points
	964 points
	4 points


Hmm since most `cv2` functions use uint8 images instead of float64 images, maybe I should rewrite all my code to be likewise.

In [10]:
# Throw out any contours with 10 or fewer points
contours_trimmed = [cnt for cnt in contours if len(cnt) > 10]
print(f'# of remaining contours: {len(contours_trimmed)}')
for cnt in contours_trimmed:
    print(f'\t{len(cnt)} points')

dst = img1_square.copy()
cmap = matplotlib.cm.get_cmap('tab10')
for i, cnt in enumerate(contours_trimmed):
    print(len(cnt))
    cv2.drawContours(dst, [cnt], 0, [255*x for x in cmap(i)], 3)

plt.figure(figsize=(4, 4))
plt.imshow(dst)
plt.title('Sample 1 with labeled contours (trimmed)')
plt.savefig('ContoursTrimmedSample1.png')

# of remaining contours: 3
	103 points
	97 points
	964 points
103
97
964


In [11]:
# I'm assuming the points in the contour are already ordered
# But let's plot the points in a line graph to make sure

x_list = [x[0][0] for x in contours_trimmed[0]]
y_list = [x[0][1] for x in contours_trimmed[0]]

plt.figure(figsize=(4, 4))
plt.plot(x_list)
plt.plot(y_list)
plt.plot(x_list, y_list)
plt.title('Ordered Points')
plt.savefig('OrderedPoints.png')

In [12]:
# Remove every other point (starting with first point)
contours_dashed = [cnt[1::2] for cnt in contours_trimmed]
print(f'# of remaining contours: {len(contours_trimmed)}')
for cnt in contours_dashed:
    print(f'\t{len(cnt)} points')

dst = img1_square.copy()
cmap = matplotlib.cm.get_cmap('tab10')
for i, cnt in enumerate(contours_dashed):
    print(len(cnt))
    # We can't use drawContours because the contour is no longer contiguous
    # Instead, fill in the points manually
    for point in cnt:
        x, y = point[0]
        dst[y, x, :] = [255*x for x in cmap(i)][0:-1] # only take 3 color channels, ignore alpha

plt.figure(figsize=(4, 4), dpi=300)
plt.imshow(dst)
plt.title('Sample 1 with dashed contours')
plt.savefig('ContoursDashedSample1.png')

# of remaining contours: 3
	51 points
	48 points
	482 points
51
48
482


In [13]:
# Compare dashed vs undashed
def draw_contours(img, contours):
    if img.dtype != np.uint8:
        print('Converting to uint8...')
        dst = (255*img).astype(np.uint8)
    else:
        dst = img.copy()
    cmap = matplotlib.cm.get_cmap('tab10')
    for i, cnt in enumerate(contours):
        for point in cnt:
            x, y = point[0]
            color = [255*x for x in cmap(i)][0:-1] # only take 3 color channels, ignore alpha
            dst[y, x, :] = color
    return dst

fig, axes = plt.subplots(1, 2, figsize=(8, 4), dpi=300)
axes[0].imshow(draw_contours(img1_square, contours_trimmed))
axes[0].set_title('Undashed')
axes[1].imshow(draw_contours(img1_square, contours_dashed))
axes[1].set_title('Dashed')
plt.tight_layout()
plt.savefig('DashedVsUndashed.png')

In [14]:
# How about every 3rd point? Or every 4th point?
fig, axes = plt.subplots(1, 2, figsize=(8, 4), dpi=300)
axes[0].imshow(draw_contours(img1_square, [cnt[1::3] for cnt in contours_trimmed]))
axes[0].set_title('Dashed (every 3rd point)')
axes[1].imshow(draw_contours(img1_square, [cnt[1::4] for cnt in contours_trimmed]))
axes[1].set_title('Dashed (every 4th point)')
plt.tight_layout()
plt.savefig('Dashed3rdor4th.png')

In [15]:
# Find sequence of angles in all the contours combined
thetaseq = []
for i, cnt in enumerate(contours_dashed):
    for j, point in enumerate(cnt):
        if j == 0:
            prevx, prevy = point[0]
        else:
            x, y = point[0]
            thetaseq.append(np.arctan2(y-prevy, x-prevx))
            prevx = x
            prevy = y

print(f'# thetas: {len(thetaseq)}')
print(f'max theta: {max(thetaseq)}')
print(f'min theta: {min(thetaseq)}')

# Calculate histogram of angles
bins = 20
plt.figure(figsize=(4, 4), dpi=300)
plt.hist(thetaseq, bins=bins, range=(-np.pi, np.pi))
plt.title('Histogram of angles')
plt.xlabel('Angle (radians)')
plt.ylabel('Frequency')
plt.savefig('Histogram.png')

# thetas: 578
max theta: 3.141592653589793
min theta: -2.677945044588987


In [16]:
# Polar plot of angle histogram
bins = 20
hist =  np.histogram(thetaseq, bins=bins, range=(-np.pi, np.pi))
r, theta = hist
r = np.append(r, 0) # append zero to correspond to the last angle

fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(4, 4), dpi=300)
ax.plot(theta, r)
ax.grid(True)

ax.set_title("Distribution of Angles", va='bottom')
plt.tight_layout()
plt.savefig('HistogramPolar.png')

In [29]:
# Repeat for a few more images
# First, let's define a few functions to automate all the steps
def preprocess(img, desired_size=256, fill_color=[255, 255, 255], threshold=0.5, invert=True):
    img_square = make_square(img, desired_size=desired_size, fill_color=fill_color)
    img_bin = binarize(img_square, threshold=threshold, invert=invert)
    return img_bin

def contour_analysis(img, n=2, trim_points=10, bins=20, verbose=False):
    # Find all contours
    contours, hierarchy = cv2.findContours(img, cv2.RETR_TREE, cv2.CHAIN_APPROX_NONE)
    if verbose:
        print(f'# of contours: {len(contours)}')
        print('# of points in each contour:')
        for cnt in contours:
            print(f'\t{len(cnt)} points')

    # Remove contours with too few points
    contours_trimmed = [cnt for cnt in contours if len(cnt) > trim_points]
    if verbose:
        print(f'Trimming contours with fewer than {trim_points} points...')
        print(f'# of remaining contours: {len(contours_trimmed)}')
        for cnt in contours_trimmed:
            print(f'\t{len(cnt)} points')

    # Create dashed contours by keeping every nth point
    assert(n>=2)
    contours_dashed = [cnt[1::n] for cnt in contours_trimmed]
    if verbose:
        print(f'Taking every {n}th point to get dashed contour...')
        for cnt in contours_dashed:
            print(f'\t{len(cnt)} points')
            
    # Find angles between adjacent points in the contour
    thetaseq = []
    for i, cnt in enumerate(contours_dashed):
        for j, point in enumerate(cnt):
            if j == 0:
                prevx, prevy = point[0]
            else:
                x, y = point[0]
                thetaseq.append(np.arctan2(y-prevy, x-prevx))
                prevx = x
                prevy = y
    
    if verbose:
        print(f'# thetas: {len(thetaseq)}')
        print(f'max theta: {max(thetaseq)}')
        print(f'min theta: {min(thetaseq)}')

    hist =  np.histogram(thetaseq, bins=bins, range=(-np.pi, np.pi))
    return hist

def plot_angles(hist, ax=None, title='Distribution of angles'):
    r, theta = hist
    r = np.append(r, 0) # append zero to correspond to the last angle

    if ax is None:
        fig, ax = plt.subplots(subplot_kw={'projection': 'polar'}, figsize=(4, 4), dpi=300)

    ax.plot(theta, r)
    ax.grid(True)

    ax.set_title(title, va='bottom')
    plt.tight_layout()

In [37]:
plt.imshow(preprocess(img3), cmap='gray')

In [38]:
contour_analysis(preprocess(img3), verbose=True)

# of contours: 63
# of points in each contour:
	2 points
	2 points
	44 points
	7 points
	75 points
	4 points
	2 points
	197 points
	2 points
	20 points
	14 points
	9 points
	18 points
	2 points
	43 points
	1 points
	10 points
	4 points
	16 points
	47 points
	21 points
	13 points
	10 points
	6 points
	1 points
	49 points
	36 points
	13 points
	2 points
	42 points
	44 points
	77 points
	54 points
	2 points
	12 points
	69 points
	17 points
	115 points
	23 points
	9 points
	475 points
	15 points
	8 points
	4 points
	4 points
	8 points
	4 points
	4 points
	145 points
	6 points
	172 points
	1030 points
	4 points
	7 points
	10 points
	8 points
	92 points
	55 points
	8 points
	73 points
	4 points
	4 points
	5 points
Trimming contours with fewer than 10 points...
# of remaining contours: 31
	44 points
	75 points
	197 points
	20 points
	14 points
	18 points
	43 points
	16 points
	47 points
	21 points
	13 points
	49 points
	36 points
	13 points
	42 points
	44 points
	77 points
	54 points
	12 poin

(array([  0,  76,  24,  58,   0, 171, 102,  41,  45,   0, 244,  72,  43,
         45,   0, 181,  95,  29,  49, 243]),
 array([-3.14159265, -2.82743339, -2.51327412, -2.19911486, -1.88495559,
        -1.57079633, -1.25663706, -0.9424778 , -0.62831853, -0.31415927,
         0.        ,  0.31415927,  0.62831853,  0.9424778 ,  1.25663706,
         1.57079633,  1.88495559,  2.19911486,  2.51327412,  2.82743339,
         3.14159265]))

In [39]:
contour_analysis(preprocess(img1), verbose=True)

# of contours: 5
# of points in each contour:
	1 points
	103 points
	97 points
	964 points
	4 points
Trimming contours with fewer than 10 points...
# of remaining contours: 3
	103 points
	97 points
	964 points
Taking every 2th point to get dashed contour...
	51 points
	48 points
	482 points
# thetas: 578
max theta: 3.141592653589793
min theta: -2.677945044588987


(array([ 0, 30, 11, 22,  0, 45, 54, 36, 41,  0, 52, 22, 10, 35,  0, 47, 47,
        31, 43, 52]),
 array([-3.14159265, -2.82743339, -2.51327412, -2.19911486, -1.88495559,
        -1.57079633, -1.25663706, -0.9424778 , -0.62831853, -0.31415927,
         0.        ,  0.31415927,  0.62831853,  0.9424778 ,  1.25663706,
         1.57079633,  1.88495559,  2.19911486,  2.51327412,  2.82743339,
         3.14159265]))

In [42]:
fig = plt.figure(figsize=(12, 4))
ax1 = plt.subplot(131, projection='polar')
ax2 = plt.subplot(132, projection='polar')
ax3 = plt.subplot(133, projection='polar')

plot_angles(contour_analysis(preprocess(img1)), ax=ax1, title='img1')
plot_angles(contour_analysis(preprocess(img2)), ax=ax2, title='img2')
plot_angles(contour_analysis(preprocess(img3)), ax=ax3, title='img3')
plt.savefig('3SamplesPolar.png')

# Contour Statistics
Treating each images's histogram as a sample of the random variable $X$, estimate:
* Mean vector $\mu = E[X]$
* Covariance matrix $K = E[(X-\mu)(X-\mu)^T] $
r
